# Child Mind Institute — Problematic Internet Use

## Team notes

from Célie, 11/2:

- Hi Anusha! I will label my work with a comment and my initials (CP).
- You can link the Kaggle notebook to your GitHub from within Kaggle.
- If using GitHub, save data locally in file kaggle_data (file is ignored by git)
- If using Kaggle notebook, data is located in: /kaggle/input/child-mind-institute-problematic-internet-use/
- I am using my local directory but if you are using Kaggle, you'll need to switch the commented out code.


In [1]:
# CP: Show available files

# CP: If using local directory
# import os
# for dirname, _, filenames in os.walk('kaggle_data'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# CP: If using Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
/kaggle/input/child-mind-institute-problematic-internet-use/train.csv
/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=00115b9f/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=001f3379/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0745c390/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=eaab7a96/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=8ec2cc63/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=b2987a65/part-0.parquet
/kaggle/input/child-mind-institute-problematic-intern

In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import (
    AdaBoostClassifier,  # Adaptive Boosting
    GradientBoostingClassifier,  
    VotingClassifier  
)
from sklearn.tree import DecisionTreeClassifier  
from xgboost import XGBClassifier  

####AG###

# CP
import numpy as np
import pandas as pd

In [3]:
# CP: Load data
# CP: If using local directory
# train = pd.read_csv('kaggle_data/train.csv')
# test = pd.read_csv('kaggle_data/test.csv')

# If using Kaggle
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train.head(20)

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.00,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.00,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.50,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.00,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.50,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
7,0068a485,Fall,10,1,NaN,NaN,Fall,16.861286,59.25,84.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,2.0,NaN
8,0069fbed,Summer,15,0,NaN,NaN,Spring,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.0,NaN
9,0083e397,Summer,19,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
####AG###

# Children Global Assessment Scale(CGAS) score is used to check the general 
#functioning level of children typically in the range of 1-100 

#there are null values in every column except for age and basic season up until row 20


In [6]:
# CP: Show all rows when displaying data
#pd.set_option('display.max_rows', None)

# CP: Load data dictionary
# data_dict = pd.read_csv('kaggle_data/data_dictionary.csv') # CP: Local
#data_dict = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv') # CP: Kaggle

#data_dict

In [7]:
# CP: Explore data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 82 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null    floa

In [8]:
# CP: Explore data
train.shape

(3960, 82)

In [9]:
# CP: Explore data
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Basic_Demos-Age,3960.0,10.433586,3.574648,5.0,8.00000,10.000000,13.000000,22.000000
Basic_Demos-Sex,3960.0,0.372727,0.483591,0.0,0.00000,0.000000,1.000000,1.000000
CGAS-CGAS_Score,2421.0,65.454771,22.341862,25.0,59.00000,65.000000,75.000000,999.000000
Physical-BMI,3022.0,19.331929,5.113934,0.0,15.86935,17.937682,21.571244,59.132048
Physical-Height,3027.0,55.946713,7.473764,33.0,50.00000,55.000000,62.000000,78.500000
...,...,...,...,...,...,...,...,...
PCIAT-PCIAT_Total,2736.0,27.896199,20.338853,0.0,12.00000,26.000000,41.000000,93.000000
SDS-SDS_Total_Raw,2609.0,41.088923,10.427433,17.0,33.00000,39.000000,46.000000,96.000000
SDS-SDS_Total_T,2606.0,57.763622,13.196091,38.0,47.00000,55.000000,64.000000,100.000000
PreInt_EduHx-computerinternet_hoursday,3301.0,1.060588,1.094875,0.0,0.00000,1.000000,2.000000,3.000000


In [10]:
# CP: Check target values
train['sii'].value_counts()

sii
0.0    1594
1.0     730
2.0     378
3.0      34
Name: count, dtype: int64

In [11]:
# CP: Check missing target values
train['sii'].isnull().sum()

from sklearn.preprocessing import StandardScaler, LabelEncoder  
from sklearn.impute import SimpleImputer 

# Explicitly enable IterativeImputer 
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer  

In [12]:
def handle_missing_values(train, test):
# Ensure both DataFrames have the same columns
    common_columns = set(train.columns).intersection(set(test.columns))
    train = train[list(common_columns)].copy()
    test = test[list(common_columns)].copy()

    # Separate numeric and categorical columns
    num_cols = train.select_dtypes(include=["number"]).columns
    cat_cols = train.select_dtypes(exclude=["number"]).columns

    # Impute missing values for numeric columns
    num_imputer = SimpleImputer(strategy="mean")
    train[num_cols] = num_imputer.fit_transform(train[num_cols])
    test[num_cols] = num_imputer.transform(test[num_cols])

    # Impute missing values for categorical columns
    cat_imputer = SimpleImputer(strategy="most_frequent")
    train[cat_cols] = cat_imputer.fit_transform(train[cat_cols])
    test[cat_cols] = cat_imputer.transform(test[cat_cols])

    return train, test

In [13]:
#Cleaning 
try:
    # Check columns before cleaning
    print("Train DataFrame columns:", train.columns.tolist())
    print("Test DataFrame columns:", test.columns.tolist())

    # Clean missing values
    train_df_cleaned, test_df_cleaned = handle_missing_values(train, test)

    # Add the 'sii' column back to `train_df_cleaned`
    if 'sii' in train.columns:
        train_df_cleaned['sii'] = train['sii']

    # Remove rows with null values in the 'sii' column
    train_df_cleaned = train_df_cleaned.dropna(subset=['sii'])

    # Rename train_df_cleaned back to train_df
    train = train_df_cleaned
    test = test_df_cleaned

    # Display cleaned data
    print("\nCleaned Train DataFrame")
    print(train.info())
    print("\nCleaned Test DataFrame:")
    print(test_df_cleaned.info())

except Exception as e:
    print(f"Error occurred: {e}")

Train DataFrame columns: ['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PA

In [14]:
#Checking for null val
print(train['sii'].isnull().sum())

0


In [15]:
X_train = train.drop(columns=['sii']) 
y_train = train['sii']

In [16]:
#Preprocessing the test data 

X_test = test.drop(columns=['sii'], errors='ignore')  

# One-hot encode categorical features 
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

#train and test data have the same columns after one-hot encoding
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

In [17]:
 if y_train.dtype == 'object':
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
elif y_train.dtype == 'float' or not np.issubdtype(y_train.dtype, np.integer):
    y_train = y_train.astype(int)

In [18]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [19]:
#Split into training and validation sets 
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)


In [20]:
#Training 
#Initializing the models 
logreg_model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=200)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
adaboost_model = AdaBoostClassifier(n_estimators=100, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)

In [21]:
#Train Voting Classifier 
voting_clf = VotingClassifier(
    estimators=[  # List of individual models
        ('logreg', logreg_model),
        ('rf', rf_model),
        ('adaboost', adaboost_model),
        ('gb', gb_model),
        ('dt', dt_model),
        ('xgb', xgb_model)
    ],
    voting='hard'  # Use hard voting for classification
)

In [22]:
voting_clf.fit(X_train_split, y_train_split)

VotingClassifier(estimators=[('logreg',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial')),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('adaboost',
                              AdaBoostClassifier(n_estimators=100,
                                                 random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('dt',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_sc...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))])

In [23]:
from sklearn.metrics import (
    accuracy_score,  
    precision_score,  
    recall_score, 
    f1_score, 
    roc_auc_score,  
    confusion_matrix  
)
y_valid_pred = voting_clf.predict(X_valid_split)
validation_accuracy = accuracy_score(y_valid_split, y_valid_pred)
print(f"Validation Accuracy (Voting Classifier): {validation_accuracy:.4f}")

#Make Predictions on Test Set

test_predictions = voting_clf.predict(X_test_scaled)

submission_df_voting = pd.DataFrame({'id': test['id'], 'sii': test_predictions})
submission_df_voting.to_csv('submission.csv', index=False)

print(submission_df_voting.head(10))  

Validation Accuracy (Voting Classifier): 0.9818
         id  sii
0  00008ff9    2
1  000fd460    0
2  00105258    0
3  00115b9f    1
4  0016bb22    0
5  001f3379    1
6  0038ba98    0
7  0068a485    0
8  0069fbed    0
9  0083e397    0
